# CODE-DE/EO-Lab tutorial

<div style="text-align: right"><i> Beginner </i></div>

<center><h1> CODE-DE/EO-LAB Sentinel-2 Bildsementierung mit Jupyter-Lab und Python.  </h1></center>
Wir verwenden den Zugang vom Jupyter-Lab um auf Sentinel-2 Daten zugreifen zu können. Es wird eine Sentinel-2 Szene verwendet. Die 10m Bänder werden lokal heruntergeladen und es wird ein RGB Subset erstellt.

Dieses Subset wird segemtiert um homogene Objekte zur Bildklassifikation zu erhalten, siehe z.B. <a href="https://doi.org/10.3390/rs8040312">https://doi.org/10.3390/rs8040312</a> als Anwendungsbeispiel. <br> 

Die Segmentierung wird als Vektor-File im GeoJSON Format exportiert. 
Siehe auch:  <a href="https://scikit-image.org/docs/stable/api/skimage.segmentation.html">scikit-image</a>
<br>

**General Note 1**: Ausführung der Zellen durch pressen des <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button vom top MENU (oder `Shift` + `Enter`).
<br>
**General Note 2**: Falls der Kern nich mehr arbeitet, im the top MENU, klicke <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Dann, im top MENU, clicke  "Run" aund wähle "Run All Above Selected Cell".
<br>
**General Note 3**: Schauen Sie sich im  <a href="https://forum.code-de.org/de/">CODE-DE/EOLab FORUM</a> um oder Kontaktieren Sie den Support! 
<br>



In [ ]:
!pip install rasterio
!pip install numpy
!pip install ogr
!pip install scikit-image
!pip install geopandas
!pip install boto3
!pip install matplotlib
import numpy as np
import ogr
from skimage import exposure
from skimage.segmentation import quickshift
from skimage.segmentation import slic
import geopandas as gpd
from rasterio import plot
import boto3
import botocore
import rasterio
import matplotlib.pyplot as plt
import os
import shapely
import pandas as pd

### Connect to the cloud and get an image

In [ ]:
# access CODE-DE S3 endpoint (setup the keys in the "S3 access credentials" section in your CODE-DE profile)
access_key='<your access key>'
secret_key='<your secret key>'
host='http://data.cloud.code-de.org'
s3=boto3.client('s3',aws_access_key_id=access_key, aws_secret_access_key=secret_key, endpoint_url=host,)
print('Connecting to CODE-DE')

prefix=  s3.list_objects(Delimiter='/', Bucket="CODEDE", Prefix='Sentinel-2/MSI/L2A/2021/10/24/S2B_MSIL2A_20211024T104029_N0301_R008_T31UGS_20211024T122748.SAFE/GRANULE/',MaxKeys=30000)['CommonPrefixes'][0]['Prefix']
prefix=prefix+"IMG_DATA/R10m/"

print('Downloading files')
fileList=[]
i=0
for content in s3.list_objects(Delimiter='/', Bucket="CODEDE", Prefix=prefix)['Contents']:
         if content['Key'].endswith('.jp2'): # filter by .jp2 suffix
                object_name = content['Key']
                file_name = object_name.split('/')[-1] # assume last part as a target filename
                if i in [2,3,4,5]:
                  print(file_name)
                  fileList.append(file_name)
                  s3.download_file("CODEDE", object_name, file_name)                  
         i=i+1    

### Erstellung eines Bild-Stack und Schreiben eines Subsets

In [ ]:
# import der einzelnen Bänder
win = rasterio.windows.Window(0, 0, 500, 500)
band2 = rasterio.open(fileList[0], driver='JP2OpenJPEG') #blue
band3 = rasterio.open(fileList[1], driver='JP2OpenJPEG') #green
band4 = rasterio.open(fileList[2], driver='JP2OpenJPEG') #red
band8 = rasterio.open(fileList[3], driver='JP2OpenJPEG') #nir

# export als eine Bilddatei
S2File='Sentinel2Img.tiff'
ColorImg = rasterio.open(S2File,'w',driver='Gtiff',
                         width=win.width, height=win.width,
                         count=4,
                         crs=band4.crs,
                         transform=band4.window_transform(win),
                         dtype=band4.dtypes[0])

ColorImg.write(band2.read(1,window=win),1) #blue
ColorImg.write(band3.read(1,window=win),2) #green
ColorImg.write(band4.read(1,window=win),3) #red
ColorImg.write(band8.read(1,window=win),4) #nir
ColorImg.close()
print('Schreiben der  RGB Datei beendet') 
for h in fileList:
  print("Lösche: "+h)  
  os.remove(h)   #cleaning up


### Nun zur Segmentierung mit scikit-image und rasterio. 

In [ ]:
# Die Bänder werden als Liste benötigt...
band_data = []
band_data.append(rasterio.open(S2File).read(1,window=win))
band_data.append(rasterio.open(S2File).read(2,window=win))
band_data.append(rasterio.open(S2File).read(3,window=win))
band_data.append(rasterio.open(S2File).read(4,window=win))
band_data = np.dstack(band_data)
print("Re-Skalierung..")
img = exposure.rescale_intensity(band_data)
print("Starte die Segementation..")
# Segmentation -  Optionen mit quickshift oder slic. Testen und Anpassung der Parameter nötig!
segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
#segments = slic(img, n_segments=500000, compactness=0.1)
print('Segementation beendet') #, time.time() - seg_start
 
# save segments to raster
OutSeg = 'segments_quickshift.tif'
out = rasterio.open(OutSeg,'w',driver='Gtiff',
                         width=win.width, height=win.width,
                         count=1,
                         crs=band4.crs,
                         transform=band4.window_transform(win),
                         dtype=band4.dtypes[0]
                         )
out.write(segments,1) 

In [ ]:
#Funktion um eine Farbpalette mit Zufallswerten zu nutzen
def rand_cmap(nlabels, type='bright', first_color_black=True, last_color_black=False, verbose=True):
    """
    https://github.com/delestro/rand_cmap
    Creates a random colormap to be used together with matplotlib. Useful for segmentation tasks
    :param nlabels: Number of labels (size of colormap)
    :param type: 'bright' for strong colors, 'soft' for pastel colors
    :param first_color_black: Option to use first color as black, True or False
    :param last_color_black: Option to use last color as black, True or False
    :param verbose: Prints the number of labels and shows the colormap. True or False
    :return: colormap for matplotlib
    """
    from matplotlib.colors import LinearSegmentedColormap
    import colorsys
    import numpy as np


    if type not in ('bright', 'soft'):
        print ('Please choose "bright" or "soft" for type')
        return

    if verbose:
        print('Number of labels: ' + str(nlabels))

    # Generate color map for bright colors, based on hsv
    if type == 'bright':
        randHSVcolors = [(np.random.uniform(low=0.0, high=1),
                          np.random.uniform(low=0.2, high=1),
                          np.random.uniform(low=0.9, high=1)) for i in range(nlabels)]

        # Convert HSV list to RGB
        randRGBcolors = []
        for HSVcolor in randHSVcolors:
            randRGBcolors.append(colorsys.hsv_to_rgb(HSVcolor[0], HSVcolor[1], HSVcolor[2]))

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]

        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    # Generate soft pastel colors, by limiting the RGB spectrum
    if type == 'soft':
        low = 0.6
        high = 0.95
        randRGBcolors = [(np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high),
                          np.random.uniform(low=low, high=high)) for i in range(nlabels)]

        if first_color_black:
            randRGBcolors[0] = [0, 0, 0]

        if last_color_black:
            randRGBcolors[-1] = [0, 0, 0]
        random_colormap = LinearSegmentedColormap.from_list('new_map', randRGBcolors, N=nlabels)

    '''# Display colorbar
    if verbose:
        from matplotlib import colors, colorbar
        from matplotlib import pyplot as plt
        fig, ax = plt.subplots(1, 1, figsize=(15, 0.5))

        bounds = np.linspace(0, nlabels, nlabels + 1)
        norm = colors.BoundaryNorm(bounds, nlabels)

        cb = colorbar.ColorbarBase(ax, cmap=random_colormap, norm=norm, spacing='proportional', ticks=None,
                                   boundaries=bounds, format='%1i', orientation=u'horizontal')
      '''
    return random_colormap

new_cmap = rand_cmap(100, type='bright', first_color_black=True, last_color_black=False, verbose=True)
plot.show(segments, title = "Segmentation",cmap=new_cmap)

 

In [ ]:
# Hier auch die Zugehörige Bild-Datei
with rasterio.open(S2File) as img:
    dataRGB = img.read([3,2,1])
    
    plot.show(dataRGB/2500, title = "Sentinel-2 true color image")

In [ ]:
from rasterio.features import shapes
from shapely.geometry import shape

from geopandas import GeoDataFrame
from pandas import DataFrame

shapes = rasterio.features.shapes(segments.astype(np.int32),transform= band4.window_transform(win)) 
shapes
pol = list(shapes)
geom = [shapely.geometry.shape(i[0]) for i in pol]
geom = gpd.GeoSeries(geom, crs=band4.crs)
print(geom)
values = [i[1] for i in pol]
values = pd.Series(values)
print(values)
# Konvertiere zu einem GeoDataFrame
gdf = gpd.GeoDataFrame({'value': values, 'geometry': geom})
outVec='segments.geojson'   
gdf.to_file(outVec, driver='GeoJSON') 

In [ ]:
# Georeferenzierter Plot mit RGB Raster und Segmenten Überlagert
fig, ax = plt.subplots(figsize=(10, 10))
   
plot.show(dataRGB/2500, ax=ax,transform=band4.window_transform(win), title = "Sentinel-2 segmentation")
# Plot the vector data on top of the raster
vector_data = gpd.read_file(outVec)
vector_data.plot(ax=ax, facecolor='none', edgecolor='yellow')

Nun kann z.B. eine objektbasierte Bildklassifizierung vorgenommen werden, in dem die Polygone mit Felddaten verschnitten werden und über Eigenschaften der Objekte Klassifiziert werden - wie im oben zitieren Paper beschrieben.